In [4]:
from utils import *

location_id = 0 #JSON with all coordinates has multiple locations (18), pick which one to work with

num_splits_for_location = 20 #in how many splits to divide the location (20 = 20x20 grid, 100 = 100x100 grid, etc.)



In [5]:
json_path = r'data/catchments_long.json' #path to JSON file with all coordinates

#parse JSON to list of coordinates (dictionaries). See function definition for long_json_to_list()
#extract one dict form the list at id: location_id
#divide the location into a grid of num_splits_for_location x num_splits_for_location. done by corners_to_grid_by_num_splits()
#location_grid_by_num_splits: 2 locumns - latitude and longitude, num_splits_for_location^2 rows - coordinates of the grid
location_grid_by_num_splits = corners_to_grid_by_num_splits(long_json_to_list(json_path)[location_id], num_splits_for_location)


pivoted_elevation_grid_by_num_splits = parse_api_resonse_to_dataframe(elevation_api_caller(location_grid_by_num_splits)).pivot(index='latitude', columns='longitude', values='elevation')